In [17]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob
import funcs_general as fg
import funcs_weather_typing as fwt
import funcs_read_ds_cmip5 as frdc5


In [2]:
regions = ['Desert Southwest', 'Great Lakes', 'Gulf Coast', 'Mid Atlantic',
            'Mountain West', 'North Atlantic', 'Northern Plains',
            'Pacific Northwest', 'Pacific Southwest']
models = ['ACCESS1-3','CanESM2','CCSM4','MIROC5','MRI-CGCM3','NorESM1-M']
# methods = ['ICAR','ICARwest','GARD_r2','GARD_r3','GARDwest','LOCA_8th','MACA','NASA-NEX']
methods = ['ICAR','ICARwest','GARD_r2','GARD_r3','LOCA_8th','MACA','NASA-NEX']
pltmeth = ['ICAR', 'ICARwest', 'GARD_AR2', 'GARD_AR3', 'LOCA', 'MACA', 'NASA-NEX']
obsdsetlist = ['PRISM', 'CONUS404', 'Livneh', 'NLDAS', 'GMET', 'nClimGrid', 'gridMET']
nreg = len(regions)
nmod = len(models)
nmeth = len(methods)
nobs = len(obsdsetlist)
nwt = 6

In [3]:
# a = xr.open_dataset('/glade/work/nlybarger/data/WT_centroids_era5/PacificNorthwest.era5.WT.centroids.UV850.Q850.Q500.PSL.1981-2005.RAINYDAYS.nc')
data = {}
for reg in regions:
    regname = reg.replace(' ','')
    fili = glob.glob(f'/glade/work/nlybarger/data/WT_centroids_era5/{regname}/{regname}.era5.WT.centroids.*.1981-2005.RAINYDAYS.nc')
    data[reg] = xr.open_dataset(fili[0])
    

In [4]:
data[reg]

<xarray.Dataset>
Dimensions:    (WT: 6, var: 4, lat: 12, lon: 9)
Coordinates:
  * WT         (WT) int64 0 1 2 3 4 5
  * var        (var) object 'V850' 'Q850' 'Q500' 'PSL'
  * lat        (lat) int64 29 30 31 32 33 34 35 36 37 38 39 40
  * lon        (lon) int64 236 237 238 239 240 241 242 243 244
Data variables:
    centroids  (WT, var, lat, lon) float64 ...
    frequency  (WT) float64 ...
    WT_precip  (WT, lat, lon) float64 ...

In [ ]:
for reg in regions:
# for reg in ['Pacific Northwest']:
    regname = reg.replace(' ','')
    nvar = len(data[reg]['var'])
    nwt = len(data[reg]['WT'])
    fig,axes = plt.subplots(nwt,nvar+1,figsize=(15,15))
    axs = axes.flat
    # ivar = iwt = 0
    # for ax in axs:
    for iwt in range(nwt):
        for ivar in range(nvar+1):
            axind = iwt*(nvar+1)+ivar
            ax = axs[axind]
            plt.sca(ax)

            if ivar == nvar:
                fg.mapper(data[reg]['WT_precip'][iwt,:,:],data[reg]['lon'],data[reg]['lat'],vmin=0,vmax=10,title='',cm='BrBG')
            else:
                fg.mapper(data[reg]['centroids'][iwt,ivar,:,:],data[reg]['lon'],data[reg]['lat'],vmin=-1,vmax=1,title='',cm='seismic')
            if iwt==0 and ivar<nvar:
                
                plt.title(data[reg]['var'][ivar].item(), fontsize=16)
            elif ivar==nvar:
                # ax.yaxis.set_label_coords(-0.1, 0.5)
                plt.title('WT Mean\nPrecipitation (mm)', fontsize=16)
            if ivar==0:
                ax.yaxis.set_label_coords(-0.1, 0.5)
                plt.ylabel(f'WT{iwt+1}', fontsize=16)
    plt.suptitle(reg, fontsize=24)
    # plt.bbox_inches='tight'
    plt.savefig(f'/glade/work/nlybarger/images/ERA5_WT/ERA5.{regname}.centroids+precip.png',bbox_inches='tight')
                


In [11]:
regions, bucods, mask = fwt.make_buco_region_map()
varlist = fwt.get_regional_WT_varlist()

In [21]:
dspr = {}
for reg in regions:
    regname = reg.replace(' ','')
    dspr[regname] = {}
    varstr = ''
    for var in varlist[reg]:
        varstr += var + '.'
    for mod in models:
        dspr[regname][mod] = {}
        for meth in methods:
            if frdc5.check_modmeth(mod, meth, reg=reg):
                dspr[regname][mod][meth] = {}
                for wt in range(6):
                    dspr[regname][mod][meth][wt] = xr.open_dataset(f'/glade/work/nlybarger/data/WT_centroids_era5/{regname}/{regname}.{mod}.{meth}.WT{wt}.{varstr}RDprecip_alltim.nc')['pr'].mean(dim='time')


In [22]:
fg.write_pickle(dspr, '/glade/derecho/scratch/nlybarger/data/all_downscaled_mean_precip.pkl')